In [118]:
import os
import pandas as pd
from datetime import datetime


In [119]:
input_file = "/Users/usi/Desktop/Papers/Adversarial_XSS/output_code_generation/all.csv"
output_file = "/Users/usi/Desktop/Papers/Adversarial_XSS/output_code_generation/filtered.csv"

In [120]:
df = pd.read_csv(input_file)
df.head()

,Unnamed: 0,scopus_id,title,publication_name,issn,isbn,eissn,volume,page_range,cover_date,doi,citation_count,affiliation,aggregation_type,subtype_description,authors,full_text
0,0,85174619893,MIRROR: multi-objective refactoring recommenda...,Automated Software Engineering,09288910,NaN,15737535,31,NaN,2024-06-01,10.1007/s10515-023-00400-1,0,[{'name': 'Hebei University of Science and Tec...,Journal,Article,"['55816561700', '58657696800', '58611926900']",NaN
1,1,85169442919,Analysis and sensitivity study of steam explos...,Annals of Nuclear Energy,03064549,NaN,18732100,194,NaN,2023-12-15,10.1016/j.anucene.2023.110064,0,"[{'name': 'Université de Lorraine', 'city': 'N...",Journal,Article,"['58182397000', '6507999725', '56403234100', '...",https://api.elsevier.com/content/article/eid/1...
2,2,85171773078,Treatment variability and its relationships to...,Drug and Alcohol Dependence,03768716,NaN,18790046,252,NaN,2023-11-01,10.1016/j.drugalcdep.2023.110961,0,[{'name': 'Complejo Hospitalario Universitario...,Journal,Article,"['35079133300', '56429697400', '6603539875', '...",https://api.elsevier.com/content/article/eid/1...
3,3,85160310918,Electrical fire risk indexing using fuzzy Petr...,Fire Safety Journal,03797112,NaN,NaN,139,NaN,2023-08-01,10.1016/j.firesaf.2023.103817,0,[{'name': 'Instituto de Investigação e Desenvo...,Journal,Article,"['57275062700', '20434810800', '58290577300']",https://api.elsevier.com/content/article/eid/1...
4,4,85168701005,Poisoning Self-supervised Learning Based Seque...,SIGIR 2023 - Proceedings of the 46th Internati...,NaN,"[{'@_fa': 'true', '$': '9781450394086'}]",NaN,NaN,300-310,2023-07-19,10.1145/3539618.3591751,0,"[{'name': 'Wuhan University', 'city': 'Wuhan',...",Conference Proceeding,Conference Paper,"['57216590565', '57739673000', '56856235900', ...",NaN


In [121]:
len(df)

147

In [122]:
df = df.drop_duplicates(subset=["title"])
len(df)

147

In [123]:
df.aggregation_type.value_counts()

aggregation_type
Journal                  104
Conference Proceeding     32
Book Series                8
Book                       3
Name: count, dtype: int64

In [124]:
df = df.query("aggregation_type == 'Conference Proceeding' | aggregation_type == 'Journal'")
len(df)

136

In [125]:
df.subtype_description.value_counts()

subtype_description
Article             93
Conference Paper    35
Review               8
Name: count, dtype: int64

In [126]:
df = df.query("subtype_description == 'Conference Paper' | subtype_description == 'Article'")
len(df)

128

In [127]:
date_format = '%Y-%m-%d'
df['date'] = df.cover_date.map(lambda x: datetime.strptime(x,date_format))

In [128]:
df['year'] = df.date.dt.year
df['year'].value_counts()

year
2023    15
2020    13
2022    12
2021     8
2015     7
2014     7
2019     6
2016     6
2018     5
2017     4
2013     4
2011     4
2012     3
1993     3
2007     3
1998     3
2010     3
1999     3
2008     2
2006     2
1989     2
1994     2
1995     2
2009     2
2024     1
2000     1
2001     1
2002     1
2003     1
2005     1
1987     1
Name: count, dtype: int64

In [129]:
df = df.query("year >= 2017")
len(df)

64

In [130]:
df['year'].value_counts()

year
2023    15
2020    13
2022    12
2021     8
2019     6
2018     5
2017     4
2024     1
Name: count, dtype: int64

In [131]:
whitelist_papers = {
    "Proceedings of the ACM Conference on Computer and Communications Security": "CCS",
    "USENIX Security Symposium":"USENIX",
    "International Symposium on Software Testing":"ISSTA",
    "Network and Distributed System Security Symposium" : "NDSS", #0 expected
    "Proceedings - IEEE Symposium on Security and Privacy": "S&P",
    "Proceedings - International Conference on Software Engineering": "ICSE",
    "Foundations of Software Engineering": "FSE",
    "Automated Software Engineering":"ASE",
    "International Conference on Software Testing, Verification and Validation":"ICST", #occhio al workshop
    "Proceedings of the ACM Web Conference" : "WWW",
    "ACM Transactions on Information Systems" : "ACM Transactions on Information Systems",
    "IEEE Transactions on Software Engineering" : "IEEE Transactions on Software Engineering",
    "Empirical Software Engineering":"Empirical Software Engineering",
    "International Conference on Data Engineering":"ICDE"
}

In [132]:
def filter_publication(row):
    conf = row.publication_name
    for ok_conf in whitelist_papers.keys():
        if conf in ok_conf:
            return True
    return False

In [133]:
df.head()


,Unnamed: 0,scopus_id,title,publication_name,issn,isbn,eissn,volume,page_range,cover_date,doi,citation_count,affiliation,aggregation_type,subtype_description,authors,full_text,date,year
0,0,85174619893,MIRROR: multi-objective refactoring recommenda...,Automated Software Engineering,09288910,NaN,15737535,31,NaN,2024-06-01,10.1007/s10515-023-00400-1,0,[{'name': 'Hebei University of Science and Tec...,Journal,Article,"['55816561700', '58657696800', '58611926900']",NaN,2024-06-01,2024
1,1,85169442919,Analysis and sensitivity study of steam explos...,Annals of Nuclear Energy,03064549,NaN,18732100,194,NaN,2023-12-15,10.1016/j.anucene.2023.110064,0,"[{'name': 'Université de Lorraine', 'city': 'N...",Journal,Article,"['58182397000', '6507999725', '56403234100', '...",https://api.elsevier.com/content/article/eid/1...,2023-12-15,2023
2,2,85171773078,Treatment variability and its relationships to...,Drug and Alcohol Dependence,03768716,NaN,18790046,252,NaN,2023-11-01,10.1016/j.drugalcdep.2023.110961,0,[{'name': 'Complejo Hospitalario Universitario...,Journal,Article,"['35079133300', '56429697400', '6603539875', '...",https://api.elsevier.com/content/article/eid/1...,2023-11-01,2023
3,3,85160310918,Electrical fire risk indexing using fuzzy Petr...,Fire Safety Journal,03797112,NaN,NaN,139,NaN,2023-08-01,10.1016/j.firesaf.2023.103817,0,[{'name': 'Instituto de Investigação e Desenvo...,Journal,Article,"['57275062700', '20434810800', '58290577300']",https://api.elsevier.com/content/article/eid/1...,2023-08-01,2023
4,4,85168701005,Poisoning Self-supervised Learning Based Seque...,SIGIR 2023 - Proceedings of the 46th Internati...,NaN,"[{'@_fa': 'true', '$': '9781450394086'}]",NaN,NaN,300-310,2023-07-19,10.1145/3539618.3591751,0,"[{'name': 'Wuhan University', 'city': 'Wuhan',...",Conference Proceeding,Conference Paper,"['57216590565', '57739673000', '56856235900', ...",NaN,2023-07-19,2023


In [134]:
df = df[df.apply(lambda x:filter_publication(x), axis=1)]
len(df)

6

In [135]:
df['publication_acronym'] = df.publication_name.map(lambda x: whitelist_papers[x])

In [136]:
df.head()

,Unnamed: 0,scopus_id,title,publication_name,issn,isbn,eissn,volume,page_range,cover_date,doi,citation_count,affiliation,aggregation_type,subtype_description,authors,full_text,date,year,publication_acronym
0,0,85174619893,MIRROR: multi-objective refactoring recommenda...,Automated Software Engineering,09288910,NaN,15737535,31,NaN,2024-06-01,10.1007/s10515-023-00400-1,0,[{'name': 'Hebei University of Science and Tec...,Journal,Article,"['55816561700', '58657696800', '58611926900']",NaN,2024-06-01,2024,ASE
9,9,85151275497,Addressing Confounding Feature Issue for Causa...,ACM Transactions on Information Systems,10468188,NaN,15582868,41,NaN,2023-02-07,10.1145/3559757,1,"[{'name': 'Tencent', 'city': 'Shenzhen', 'coun...",Journal,Article,"['56285637300', '57218706452', '57191254399', ...",NaN,2023-02-07,2023,ACM Transactions on Information Systems
11,11,85171832780,Personalized action suggestions in low-code au...,Proceedings - International Conference on Soft...,02705257,"[{'@_fa': 'true', '$': '9798350322637'}]",NaN,NaN,346-350,2023-01-01,10.1109/ICSE-Companion58688.2023.00100,0,"[{'name': 'Microsoft Corporation', 'city': 'Re...",Conference Proceeding,Conference Paper,"['58597810900', '57198770356', '57848393800', ...",NaN,2023-01-01,2023,ICSE
12,12,85171735954,Long-term Static Analysis Rule Quality Monitor...,Proceedings - International Conference on Soft...,02705257,"[{'@_fa': 'true', '$': '9798350300376'}]",NaN,NaN,315-326,2023-01-01,10.1109/ICSE-SEIP58684.2023.00034,0,"[{'name': 'Amazon.com, Inc.', 'city': 'Seattle...",Conference Proceeding,Conference Paper,"['57210117673', '57758574000', '28168059600', ...",NaN,2023-01-01,2023,ICSE
16,16,85120058356,Detecting Continuous Integration Skip Commits ...,IEEE Transactions on Software Engineering,00985589,NaN,19393520,48,4873-4891,2022-12-01,10.1109/TSE.2021.3129165,2,"[{'name': 'École de Technologie Supérieure', '...",Journal,Article,"['57212273039', '50761492200', '55904259300']",NaN,2022-12-01,2022,IEEE Transactions on Software Engineering


In [137]:
blacklist_papers = ["workshops"]

In [138]:
df = df[["title", "publication_name","publication_acronym", "year", "doi", "full_text"]]
df.to_csv(output_file, index=False)